# Uitwerkingen opgaven data-analyse

## Opgave 1.3

We maken hier gebruik van de error function, gedefinieerd in `scipy.special.erf`. Er is ook een complementaire error function, gedefinieerd als $\mathrm{erfc} = 1 - \mathrm{erf}$. Vergeet niet de factor $\sqrt{2}$, aangezien $\mathrm{erf}$ een vereenvoudigde functie is, en het argument nog omgerekend moet worden.

De waarschijnlijkheid dat een waarde méér dan $1.23\sigma$ van het gemiddelde afligt is gegeven door:

In [1]:
from math import sqrt
from scipy.special import erf, erfc

In [2]:
erfc(1.23 / sqrt(2))

0.21869710485138394

De waarschijnlijkheid dat een waarde méér dan $2.43\sigma$ *boven* het gemiddelde ligt is gegeven door:

In [3]:
erfc(2.43 / sqrt(2)) / 2

0.0075494114163092065

De waarschijnlijkheid dat een waarde meer dan $0.5\sigma$, maar minder dan $1.5\sigma$ van het gemiddelde verwijderd is, wordt gegeven door:

In [4]:
erf(1.5 / sqrt(2)) - erf(.5 / sqrt(2))

0.4834606749142577

We willen weten binnen hoeveel standaarddeviaties van het gemiddelde de waarschijnlijkheid dat een waarde gevonden wordt gelijk is aan $50\,\%$. Gebruik hiervoor de inverse error function `erfinv`. Vergeet de factor $\sqrt{2}$ niet!

In [5]:
from scipy.special import erfinv

In [6]:
erfinv(.5) * sqrt(2)

0.67448975019608171

## Opgave 1.4

We maken gebruik van pandas, een python data-analyse pakket. We importeren de datafile:

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv('10-metingen.txt')

In [10]:
print(data)

    x
0  73
1  79
2  69
3  74
4  68
5  72
6  72
7  75
8  60
9  76


...en bekijken wat statistieken:

In [9]:
data.describe()

,x
count,10.000000
mean,71.800000
std,5.245104
min,60.000000
25%,69.750000
50%,72.500000
75%,74.750000
max,79.000000


De standaarddeviatie van het gemiddelde is:

In [15]:
x = data.x
x.std() / sqrt(x.count())

1.6586473739499636

Het resultaat van ons experiment is dus $x = 71.8 \pm 1.7$ of $x = 71.8(17)$.

Voor De kans op een volgende meting met $x\geq 75$ berekenen we eerst hoeveel standaarddeviaties de meting verwijderd is van het gemiddelde. Vervolgens berekenen we de kans op een meting hóger dan dat aantal standaarddeviaties:

In [17]:
dist = 75 - x.mean()
dist_sigma = dist / x.std()
erfc(dist_sigma / sqrt(2)) / 2

0.27090016180692866